# Geradin and Cardona test case

In [1]:
# Necessary packages and imports
import h5py as h5
import numpy as np
import configparser

## Description

This case is a simple static cantilever beam, used as a test case by Geradin and Cardona in their book Flexible Multibody Dynamics.

The beam length is $L = 5$ m and its mass per unit length is $\bar{m} = 100$ kg/m. Its cross-sectional inertia is $J = 10$ kgm. Stiffness is: $EA = 4.8e8$ N, $GA = 3.231e8$ N, $GJ = 1e6$ N, $EI = 9.346e6$ Nm.

However, the test case is formulated so that the weight of the beam is not included. Only a -600kN dead vertical tip force will be applied. This will be modelled as a lumped mass at the tip with mass = 61183 kg

In [2]:
# parameters
length = 5
m_bar = 100*0       # we don't consider the dsitributed mass
j = 10

ea = 4.8e8
ga = 3.231e8
gj = 1e6
ei = 9.346e6

tip_lumped_mass = 600e3/9.81

For this simulation, we are going to use 10 3-noded elements. 

In [3]:
num_node_elem = 3
num_elem = 10
num_node = (num_node_elem - 1)*num_elem + 1

## Setup

A complete input file needs all the following information:
+ [Nodal Coordinates](#nodal-coordinates)
+ [Connectivity information](#connectivities)
+ [Boundary Conditions](#boundary-conditions)
+ [Structural Twist](#structural-twist)
+ [Frame of Reference Definition](#frame-of-reference-definition)
+ [Beam Properties](#beam-properties-definition)
+ [Beam Number](#beam-number)

---

### Nodal coordinates

Nodal coordinates are stored in 3 different vectors: `x`, `y` and `z` of `size = num_node`:

In [4]:
x = np.linspace(0, length, num_node)  # very easy straight beam
y = np.zeros_like(x)
z = np.zeros_like(x)

### Connectivities

Connectivities are given in a [num_elem, num_node_elem] matrix. The entry for a element like `0---1---2` would be `[0, 2, 1]` (always extrema first). For a two-noded elem like `0---1`, I'd be `[0, 1]`

In [5]:
conn = np.zeros((num_elem, num_node_elem), dtype=int)
for ielem in range(num_elem):
    conn[ielem, :] = (np.ones(3,))*ielem*(num_node_elem - 1) + [0, 2, 1]

### Boundary conditions
Boundary conditions are given in a integer vector with as many elements as nodes has the beam. All the nodes have to be 0, except the **clamped** nodes (1) or the **free** ones (-1).

In [8]:
boundary_conditions = np.zeros((num_node, 1), dtype=int)
boundary_conditions[0] = 1
boundary_conditions[-1]=-1

### Structural twist

SHARPy supports pre-twist in beams, which will be independent from the aerodynamic twist.
This angle is given for every node in a vector:
`structural_twist[inode] = twist (rad)`.
In our case, this is only zeroes.

In [6]:
structural_twist = np.zeros_like(x)

### Frame of reference definition
In order to define consistently the local (element) frame of reference, a vector has to be indicated. The resulting triad will be:
+ `x vector` in the direction of the increasing node order (tangent vector)
+ `y vector` in the plane defined by the x vector and the indicated one. The normal vector is chosen normal to the x vector, belonging to this plane and in the direction of the indicated vector.
+ `z vector` is chosen to have a right-handed FoR

For our case, where the beam is straight and in the x direction, the easiest way of indicating the vector is to input [0, 1, 0] in every node.
Note that it is not necessary to give unit vectors, they will be normalised later.

In [9]:
frame_of_reference_delta = np.zeros((num_node, 3))
for inode in range(num_node):
    frame_of_reference_delta[inode, :] = [0., 1., 0.]

### Beam properties description
Beam properties are stiffness and distributed inertia. The properties are assumed constant inside an element. These two are given by `6x6` diagonal matrices for this case:

Mass:
$\mathcal{M} = \text{diag}(\bar{m}, \bar{m}, \bar{m}, J_x, J_y, J_z)$

Stiffness:
$\mathcal{K} = \text{diag}(EA, GA, GA, GJ, EI_y, EI_z)$

Please note that the unit system is arbitrary but needs to be consistent. $\bar{m}$ denotes the mass per unit length.

---
In order not to repeat these `6x6` matrices, these are stored in a "dictionary". Then, for every element, only the index of where the matrix is stored is associated. It is easier with an example in mind. Our beam has constant properties, so the mass dictionary and the stiffness dictionary will have only one entry each:

In [11]:
num_stiffness = 1
stiffness = np.zeros((num_stiffness, 6, 6))
stiffness[0, :, :] = np.diag([ea, ga, ga, gj, ei, ei])  # this stiffness matrix is the "dictionary"
elem_stiffness = np.zeros((num_elem,), dtype=int)       # Every element has an integer associated which is the first index
                                                        # of the stiffness[:,:,:] matrix

In [14]:
# same for the mass
num_mass = 1
mass = np.zeros((num_mass, 6, 6))
mass[0, :, :] = np.diag([m_bar, m_bar, m_bar, j, j, j])
elem_mass = np.zeros((num_elem,), dtype=int)

### Beam number
Beam number is useful when running multibeam simulations. Basically it is a way of grouping beams together for easier post processing. Here, all the elements belong to beam no. 0. Please note that it is the __element__ that belongs to the beam, not the node.

In [15]:
beam_number = np.zeros((num_elem, 1), dtype=int)

### Lumped masses
Lumped masses are associated with a node, and they are defined with a `3x3` inertia matrix, a mass and a position given in __local__ (element) frame of reference (see [Frame of reference definition](#frame-of-reference-definition)).

Here, we are going to add a lumped point mass at the tip of the beam. Its inertia will be zero (because it is a point mass) and the mass is going to be `600e3/9.81` (see [Problem description](#description)).

In [16]:
n_lumped_mass = 1
lumped_mass_nodes = np.array([num_node - 1], dtype=int)  # the first (and only) lumped mass is associated with the 
                                                         # num_node - 1 node, which is the last
lumped_mass = np.array([600e3/9.81])
lumped_mass_inertia = np.zeros((n_lumped_mass, 3, 3))
lumped_mass_position = np.zeros((n_lumped_mass, 3))

### Applied follower forces
There is also an option for applying static follower forces (in local frame of reference).
These are also applied over a node and expressed in local FoR. Every applied force/moment is given by 6 components: $F_x, F_y, F_z, M_x, M_y, M_z$.
Our case has none, but as an example:

In [17]:
#n_app_forces = 1
#node_app_forces = np.array([num_node - 1])
#app_forces = np.zeros((n_app_forces, 6))
#app_forces[0, :] = [0, 0, -1000, 0, 0, 0]

# if no forces:
n_app_forces = 0
node_app_forces = np.array([])
app_forces = np.zeros((n_app_forces, 6))